In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import qiskit as qk
import matplotlib.pyplot as plt
import pandas as pd
import networkx as nx

In [ ]:
G = nx.grid_2d_graph(3,3)

# node coordinates
n_nodes = G.number_of_nodes()
a = 0
b = 3
sqrtn = int(np.floor(np.sqrt(n_nodes)))
X = np.zeros((sqrtn * sqrtn, 2))
counter = 0
for i in range(0,sqrtn):
    for j in range(0,sqrtn):
        X[counter, 0] = a + (b - a) * (i - 1) / (sqrtn - 1)
        X[counter, 1] = a + (b - a) * (j - 1) / (sqrtn - 1)
        counter += 1
pos = {}
for node in range(0,n_nodes):
    pos[node] = X[node,:]

# graph incidence
G = nx.convert_node_labels_to_integers(G)  
B = nx.incidence_matrix(G, oriented=True).todense().T
root = 3
b = B[:,root]
print('root degree',np.sum(b*b, axis=0))
X = np.delete(B, root, 1) # remove col of root
# number of STs
print("number of STs",np.linalg.det(X.T@X))
# edge-node incidence

color_map = []
for node in G:
    if node == root:
        color_map.append('blue')
    else: 
        color_map.append('green')   
        
nx.draw(G, pos=pos, node_color=color_map , with_labels = True,font_size=20,node_size=500)


In [ ]:
from utilities.circuits_amplitude_amplification import sample_Clifford_circuit

# number of applications of Q (Grover's operator)
m = 3
num_shots = 2
samples = sample_Clifford_circuit(X,num_shots,m)
sample_DPP = samples[0,:]
print("number of edges in sample: ", sample_DPP.sum())

G_sp = nx.empty_graph(n_nodes)
it = 0
for e in G.edges:
    if sample_DPP[it] == 1:
        G_sp.add_edge(e[0],e[1])
    it = it + 1
    
nx.draw(G_sp, pos=pos, node_color=color_map , with_labels = True,font_size=20,node_size=500)